In [275]:
import os
import numpy as np
import math
import sys
from time import time
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
import matplotlib.pyplot as plt

In [276]:
dataloader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
        "data/MNIST",
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5, ), std=(0.5,))])
    ),
    batch_size=128,
    shuffle=True,
)

In [277]:
class Opt(object):
    n_epochs = 40
    Diters = 5
    batchSize = 128
    lr = 0.00005
    n_cpu = 1
    latent_dim = 100
    imageSize = 32
    channels = 1
    n_critic = 5
    experiment = "experiments"
    sample_interval = 400

In [278]:
#Weight initialization
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [282]:
class Generator(nn.Module):
    def __init__(self, in_channel = 100,
             out_channel = 1,
             feature_map = 128,
             kernal_size = 3,
             image_size = 28):
        super(Generator, self).__init__()
        
        #Upsample random variable
        layers = nn.ModuleList()
        layers += [nn.ConvTranspose2d(in_channel, feature_map, 4, 1, 0)]
        layers += [nn.BatchNorm2d(feature_map)]
        layers += [nn.ReLU(True)]
        
        size = 4
        
        layers += [nn.ConvTranspose2d(feature_map, feature_map // 2, kernal_size, 2, 1, bias = False)]
        layers += [nn.BatchNorm2d(feature_map // 2)]
        layers += [nn.ReLU(True)]
        feature_map = feature_map // 2
        size = size * 2
        
        #Main G structure
        while size < image_size // 2:
            layers += [nn.ConvTranspose2d(feature_map, feature_map // 2, 4, 2, 1, bias = False)]
            layers += [nn.BatchNorm2d(feature_map // 2)]
            layers += [nn.ReLU(True)]
            feature_map = feature_map // 2
            size = size * 2
        
        #Final layer
        layers += [nn.ConvTranspose2d(feature_map, out_channel, 4, 2, 1, bias = False)]
        layers += [nn.Tanh()]
        
        self.g = nn.Sequential(*layers)
        
    def forward(self, z):
        return self.g(z)
        
            

In [283]:
class Discriminator(nn.Module):
    def __init__(self, in_channel = 1,
             out_channel = 1,
             feature_map = 32,
             kernal_size = 3,
             image_size = 28):
        super(Discriminator, self).__init__()
        
        #First layer
        layers = nn.ModuleList()
        layers += [nn.Conv2d(in_channel, feature_map, 4, 2, 1)]
        layers += [nn.BatchNorm2d(feature_map)]
        layers += [nn.LeakyReLU(0.2, inplace=True)]
        
        size = image_size / 2
        
        #Main D structure
        while size > 8:
            layers += [nn.Conv2d(feature_map, feature_map * 2, 4, 2, 1, bias = False)]
            layers += [nn.BatchNorm2d(feature_map * 2)]
            layers += [nn.LeakyReLU(0.2, inplace=True)]
            feature_map = feature_map * 2
            size = size / 2
            
        layers += [nn.Conv2d(feature_map, feature_map * 2, kernal_size, 2, 1, bias = False)]
        layers += [nn.BatchNorm2d(feature_map * 2)]
        layers += [nn.LeakyReLU(0.2, inplace=True)]
        feature_map = feature_map * 2
        size = size / 2
        
        #Final layer
        layers += [nn.Conv2d(feature_map, out_channel, 4, 1, 0, bias = False)]
        
        self.d = nn.Sequential(*layers)
        
    def forward(self, image):
        return self.d(image)  

In [284]:
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")
generator = Generator().to(device)
discriminator = Discriminator().to(device)

In [285]:
opt = Opt()
cuda = True if torch.cuda.is_available() else False

generator_optimizer = torch.optim.RMSprop(generator.parameters(), lr=opt.lr)
discriminator_optimizer = torch.optim.RMSprop(discriminator.parameters(), lr=opt.lr)
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
batches_done = 0
saved_imgs = []
input = torch.FloatTensor(opt.batchSize, 1, opt.imageSize, opt.imageSize)
noise = torch.FloatTensor(opt.batchSize, opt.latent_dim)
fixed_noise = torch.FloatTensor(opt.batchSize, opt.latent_dim).normal_(0, 1)
one = torch.FloatTensor([1])
mone = one * -1

gen_iterations = 0
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        real_imgs = Variable(imgs.type(Tensor))
        real_imgs = real_imgs.to(device)
        for _ in range(opt.Diters):
            noise = torch.randn(opt.batchSize,opt.latent_dim, 1,1,device=device)
            fake = generator(noise).detach()
            errD_fake = torch.mean(discriminator(fake))
            errD_real = torch.mean(discriminator(real_imgs))
            errD = errD_fake - errD_real
            discriminator.zero_grad()
            errD.backward()
            discriminator_optimizer.step()
            for p in discriminator.parameters():
                p.data.clamp_(-0.01, 0.01)
                
        gen_fake = generator(noise)
        errG = -torch.mean(discriminator(gen_fake))        
        generator.zero_grad()
        errG.backward()
        generator_optimizer.step()
        gen_iterations += 1

        print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f'
            % (epoch, opt.n_epochs, i, len(dataloader), gen_iterations,
            errD.item(), errG.item(), errD_real.item(), errD_fake.item()))
        if gen_iterations % 300 == 0:
            real_imgs = real_imgs.mul(0.5).add(0.5)
            save_image(real_imgs, '{0}/real_samples.png'.format(opt.experiment))
            with torch.no_grad():
                fake = generator(noise)
            fake.data = fake.data.mul(0.5).add(0.5)
            save_image(fake.data, '{0}/fake_samples_{1}.png'.format(opt.experiment, gen_iterations))

[0/40][5/469][1] Loss_D: -0.001752 Loss_G: 0.000785 Loss_D_real: 0.001042 Loss_D_fake -0.000711
[0/40][6/469][2] Loss_D: -0.001657 Loss_G: 0.000266 Loss_D_real: 0.001434 Loss_D_fake -0.000223
[0/40][7/469][3] Loss_D: -0.001777 Loss_G: 0.000147 Loss_D_real: 0.001673 Loss_D_fake -0.000104
[0/40][8/469][4] Loss_D: -0.001932 Loss_G: 0.000093 Loss_D_real: 0.001899 Loss_D_fake -0.000032
[0/40][9/469][5] Loss_D: -0.002041 Loss_G: -0.000010 Loss_D_real: 0.002119 Loss_D_fake 0.000078
[0/40][10/469][6] Loss_D: -0.002196 Loss_G: -0.000008 Loss_D_real: 0.002256 Loss_D_fake 0.000060
[0/40][11/469][7] Loss_D: -0.002428 Loss_G: 0.000106 Loss_D_real: 0.002382 Loss_D_fake -0.000046
[0/40][12/469][8] Loss_D: -0.002724 Loss_G: 0.000047 Loss_D_real: 0.002710 Loss_D_fake -0.000014
[0/40][13/469][9] Loss_D: -0.002884 Loss_G: 0.000083 Loss_D_real: 0.002850 Loss_D_fake -0.000034
[0/40][14/469][10] Loss_D: -0.003041 Loss_G: 0.000094 Loss_D_real: 0.003005 Loss_D_fake -0.000036
[0/40][15/469][11] Loss_D: -0.0035

[0/40][89/469][85] Loss_D: -0.113916 Loss_G: 0.058089 Loss_D_real: 0.060896 Loss_D_fake -0.053020
[0/40][90/469][86] Loss_D: -0.119285 Loss_G: 0.055532 Loss_D_real: 0.060271 Loss_D_fake -0.059015
[0/40][91/469][87] Loss_D: -0.118593 Loss_G: 0.060337 Loss_D_real: 0.062853 Loss_D_fake -0.055740
[0/40][92/469][88] Loss_D: -0.122157 Loss_G: 0.057823 Loss_D_real: 0.060627 Loss_D_fake -0.061530
[0/40][93/469][89] Loss_D: -0.124164 Loss_G: 0.062717 Loss_D_real: 0.065146 Loss_D_fake -0.059018
[0/40][94/469][90] Loss_D: -0.128004 Loss_G: 0.059880 Loss_D_real: 0.063216 Loss_D_fake -0.064788
[0/40][95/469][91] Loss_D: -0.125843 Loss_G: 0.065970 Loss_D_real: 0.065343 Loss_D_fake -0.060500
[0/40][96/469][92] Loss_D: -0.131885 Loss_G: 0.067568 Loss_D_real: 0.067540 Loss_D_fake -0.064346
[0/40][97/469][93] Loss_D: -0.134074 Loss_G: 0.067315 Loss_D_real: 0.070312 Loss_D_fake -0.063762
[0/40][98/469][94] Loss_D: -0.131813 Loss_G: 0.062078 Loss_D_real: 0.063995 Loss_D_fake -0.067818
[0/40][99/469][95] L

[0/40][172/469][168] Loss_D: -0.178942 Loss_G: 0.079977 Loss_D_real: 0.074311 Loss_D_fake -0.104631
[0/40][173/469][169] Loss_D: -0.183190 Loss_G: 0.102747 Loss_D_real: 0.096912 Loss_D_fake -0.086278
[0/40][174/469][170] Loss_D: -0.191651 Loss_G: 0.096320 Loss_D_real: 0.093733 Loss_D_fake -0.097918
[0/40][175/469][171] Loss_D: -0.190080 Loss_G: 0.101560 Loss_D_real: 0.097027 Loss_D_fake -0.093053
[0/40][176/469][172] Loss_D: -0.189859 Loss_G: 0.091566 Loss_D_real: 0.087281 Loss_D_fake -0.102578
[0/40][177/469][173] Loss_D: -0.182240 Loss_G: 0.103625 Loss_D_real: 0.100477 Loss_D_fake -0.081763
[0/40][178/469][174] Loss_D: -0.184952 Loss_G: 0.086276 Loss_D_real: 0.083149 Loss_D_fake -0.101803
[0/40][179/469][175] Loss_D: -0.188245 Loss_G: 0.105744 Loss_D_real: 0.101634 Loss_D_fake -0.086611
[0/40][180/469][176] Loss_D: -0.189702 Loss_G: 0.088605 Loss_D_real: 0.084503 Loss_D_fake -0.105199
[0/40][181/469][177] Loss_D: -0.193141 Loss_G: 0.102257 Loss_D_real: 0.099763 Loss_D_fake -0.093378


[0/40][254/469][250] Loss_D: -0.189348 Loss_G: 0.102760 Loss_D_real: 0.100263 Loss_D_fake -0.089085
[0/40][255/469][251] Loss_D: -0.188919 Loss_G: 0.090916 Loss_D_real: 0.090843 Loss_D_fake -0.098076
[0/40][256/469][252] Loss_D: -0.190614 Loss_G: 0.105279 Loss_D_real: 0.102596 Loss_D_fake -0.088017
[0/40][257/469][253] Loss_D: -0.191838 Loss_G: 0.088626 Loss_D_real: 0.091780 Loss_D_fake -0.100058
[0/40][258/469][254] Loss_D: -0.187838 Loss_G: 0.103490 Loss_D_real: 0.102783 Loss_D_fake -0.085055
[0/40][259/469][255] Loss_D: -0.190824 Loss_G: 0.086889 Loss_D_real: 0.091175 Loss_D_fake -0.099649
[0/40][260/469][256] Loss_D: -0.186178 Loss_G: 0.105228 Loss_D_real: 0.103161 Loss_D_fake -0.083017
[0/40][261/469][257] Loss_D: -0.186619 Loss_G: 0.078276 Loss_D_real: 0.083123 Loss_D_fake -0.103496
[0/40][262/469][258] Loss_D: -0.183059 Loss_G: 0.104126 Loss_D_real: 0.104654 Loss_D_fake -0.078405
[0/40][263/469][259] Loss_D: -0.189971 Loss_G: 0.081001 Loss_D_real: 0.084581 Loss_D_fake -0.105390


[0/40][336/469][332] Loss_D: -0.154709 Loss_G: 0.094745 Loss_D_real: 0.090328 Loss_D_fake -0.064381
[0/40][337/469][333] Loss_D: -0.165955 Loss_G: 0.073011 Loss_D_real: 0.067292 Loss_D_fake -0.098663
[0/40][338/469][334] Loss_D: -0.170846 Loss_G: 0.098159 Loss_D_real: 0.092961 Loss_D_fake -0.077885
[0/40][339/469][335] Loss_D: -0.168306 Loss_G: 0.076958 Loss_D_real: 0.072930 Loss_D_fake -0.095376
[0/40][340/469][336] Loss_D: -0.167440 Loss_G: 0.099710 Loss_D_real: 0.092486 Loss_D_fake -0.074954
[0/40][341/469][337] Loss_D: -0.169887 Loss_G: 0.078008 Loss_D_real: 0.072277 Loss_D_fake -0.097609
[0/40][342/469][338] Loss_D: -0.160708 Loss_G: 0.094473 Loss_D_real: 0.090410 Loss_D_fake -0.070299
[0/40][343/469][339] Loss_D: -0.164667 Loss_G: 0.072373 Loss_D_real: 0.067775 Loss_D_fake -0.096892
[0/40][344/469][340] Loss_D: -0.160794 Loss_G: 0.097832 Loss_D_real: 0.091958 Loss_D_fake -0.068836
[0/40][345/469][341] Loss_D: -0.169656 Loss_G: 0.079996 Loss_D_real: 0.073574 Loss_D_fake -0.096082


...

[1/40][443/469][908] Loss_D: -0.069138 Loss_G: 0.053413 Loss_D_real: 0.046073 Loss_D_fake -0.023065
[1/40][444/469][909] Loss_D: -0.066988 Loss_G: 0.022770 Loss_D_real: 0.016741 Loss_D_fake -0.050246
[1/40][445/469][910] Loss_D: -0.069118 Loss_G: 0.054014 Loss_D_real: 0.040078 Loss_D_fake -0.029039
[1/40][446/469][911] Loss_D: -0.068045 Loss_G: 0.025134 Loss_D_real: 0.017413 Loss_D_fake -0.050632
[1/40][447/469][912] Loss_D: -0.066398 Loss_G: 0.061711 Loss_D_real: 0.049514 Loss_D_fake -0.016885
[1/40][448/469][913] Loss_D: -0.070874 Loss_G: 0.028311 Loss_D_real: 0.014509 Loss_D_fake -0.056364
[1/40][449/469][914] Loss_D: -0.068854 Loss_G: 0.047540 Loss_D_real: 0.032582 Loss_D_fake -0.036272
[1/40][450/469][915] Loss_D: -0.069131 Loss_G: 0.018001 Loss_D_real: 0.011994 Loss_D_fake -0.057138
[1/40][451/469][916] Loss_D: -0.065093 Loss_G: 0.061805 Loss_D_real: 0.047759 Loss_D_fake -0.017334
[1/40][452/469][917] Loss_D: -0.070975 Loss_G: 0.024366 Loss_D_real: 0.014567 Loss_D_fake -0.056408


[2/40][57/469][991] Loss_D: -0.063947 Loss_G: 0.034923 Loss_D_real: 0.026147 Loss_D_fake -0.037800
[2/40][58/469][992] Loss_D: -0.065002 Loss_G: 0.072740 Loss_D_real: 0.048598 Loss_D_fake -0.016404
[2/40][59/469][993] Loss_D: -0.062521 Loss_G: 0.011131 Loss_D_real: 0.003046 Loss_D_fake -0.059475
[2/40][60/469][994] Loss_D: -0.062968 Loss_G: 0.060133 Loss_D_real: 0.047598 Loss_D_fake -0.015370
[2/40][61/469][995] Loss_D: -0.065864 Loss_G: 0.017120 Loss_D_real: 0.007658 Loss_D_fake -0.058206
[2/40][62/469][996] Loss_D: -0.062252 Loss_G: 0.066921 Loss_D_real: 0.048311 Loss_D_fake -0.013941
[2/40][63/469][997] Loss_D: -0.061336 Loss_G: 0.005485 Loss_D_real: -0.001728 Loss_D_fake -0.063064
[2/40][64/469][998] Loss_D: -0.061641 Loss_G: 0.052405 Loss_D_real: 0.046133 Loss_D_fake -0.015508
[2/40][65/469][999] Loss_D: -0.066111 Loss_G: 0.037312 Loss_D_real: 0.019507 Loss_D_fake -0.046604
[2/40][66/469][1000] Loss_D: -0.063516 Loss_G: 0.059766 Loss_D_real: 0.040555 Loss_D_fake -0.022961
[2/40][6

[2/40][139/469][1073] Loss_D: -0.057253 Loss_G: 0.020932 Loss_D_real: 0.017019 Loss_D_fake -0.040234
[2/40][140/469][1074] Loss_D: -0.058924 Loss_G: 0.050377 Loss_D_real: 0.038753 Loss_D_fake -0.020171
[2/40][141/469][1075] Loss_D: -0.060728 Loss_G: 0.035546 Loss_D_real: 0.023146 Loss_D_fake -0.037581
[2/40][142/469][1076] Loss_D: -0.058574 Loss_G: 0.048800 Loss_D_real: 0.035408 Loss_D_fake -0.023167
[2/40][143/469][1077] Loss_D: -0.058587 Loss_G: 0.006130 Loss_D_real: 0.002124 Loss_D_fake -0.056463
[2/40][144/469][1078] Loss_D: -0.057774 Loss_G: 0.046531 Loss_D_real: 0.037412 Loss_D_fake -0.020362
[2/40][145/469][1079] Loss_D: -0.057880 Loss_G: 0.000389 Loss_D_real: -0.003256 Loss_D_fake -0.061136
[2/40][146/469][1080] Loss_D: -0.054938 Loss_G: 0.059224 Loss_D_real: 0.049903 Loss_D_fake -0.005035
[2/40][147/469][1081] Loss_D: -0.055788 Loss_G: 0.001524 Loss_D_real: -0.009592 Loss_D_fake -0.065380
[2/40][148/469][1082] Loss_D: -0.057391 Loss_G: 0.051201 Loss_D_real: 0.039874 Loss_D_fak

[2/40][220/469][1154] Loss_D: -0.052036 Loss_G: 0.003457 Loss_D_real: -0.003394 Loss_D_fake -0.055430
[2/40][221/469][1155] Loss_D: -0.051263 Loss_G: 0.056044 Loss_D_real: 0.046840 Loss_D_fake -0.004423
[2/40][222/469][1156] Loss_D: -0.049509 Loss_G: 0.015970 Loss_D_real: 0.000608 Loss_D_fake -0.048901
[2/40][223/469][1157] Loss_D: -0.052605 Loss_G: 0.047656 Loss_D_real: 0.035347 Loss_D_fake -0.017258
[2/40][224/469][1158] Loss_D: -0.053390 Loss_G: 0.015622 Loss_D_real: 0.003501 Loss_D_fake -0.049890
[2/40][225/469][1159] Loss_D: -0.055805 Loss_G: 0.046881 Loss_D_real: 0.040491 Loss_D_fake -0.015314
[2/40][226/469][1160] Loss_D: -0.056552 Loss_G: 0.016817 Loss_D_real: 0.003924 Loss_D_fake -0.052628
[2/40][227/469][1161] Loss_D: -0.050854 Loss_G: 0.062440 Loss_D_real: 0.043361 Loss_D_fake -0.007492
[2/40][228/469][1162] Loss_D: -0.052456 Loss_G: 0.016717 Loss_D_real: 0.000658 Loss_D_fake -0.051798
[2/40][229/469][1163] Loss_D: -0.053884 Loss_G: 0.059119 Loss_D_real: 0.043088 Loss_D_fake

[2/40][301/469][1235] Loss_D: -0.050874 Loss_G: 0.009332 Loss_D_real: -0.004293 Loss_D_fake -0.055167
[2/40][302/469][1236] Loss_D: -0.050584 Loss_G: 0.057711 Loss_D_real: 0.040428 Loss_D_fake -0.010156
[2/40][303/469][1237] Loss_D: -0.052541 Loss_G: 0.013717 Loss_D_real: 0.000564 Loss_D_fake -0.051976
[2/40][304/469][1238] Loss_D: -0.053134 Loss_G: 0.052513 Loss_D_real: 0.039616 Loss_D_fake -0.013519
[2/40][305/469][1239] Loss_D: -0.053142 Loss_G: 0.031377 Loss_D_real: 0.014850 Loss_D_fake -0.038292
[2/40][306/469][1240] Loss_D: -0.052217 Loss_G: 0.035708 Loss_D_real: 0.023459 Loss_D_fake -0.028758
[2/40][307/469][1241] Loss_D: -0.047416 Loss_G: 0.028683 Loss_D_real: 0.015110 Loss_D_fake -0.032307
[2/40][308/469][1242] Loss_D: -0.050742 Loss_G: 0.010783 Loss_D_real: 0.005874 Loss_D_fake -0.044868
[2/40][309/469][1243] Loss_D: -0.049531 Loss_G: 0.058762 Loss_D_real: 0.051088 Loss_D_fake 0.001556
[2/40][310/469][1244] Loss_D: -0.048264 Loss_G: 0.009181 Loss_D_real: -0.004845 Loss_D_fake